# Нейросети в обработке текста

Нейросети могут выделить сложные паттерны — взаимоотношения в данных. Для текстов, паттерны проявляются, как именно слова употребляются вместе, как построены фразы. Другими словами, нейросети нужны для того, чтобы представить контекст в виде математического объекта — вектора или матрицы. 

### Основные блоки

1. **Свёрточные блоки** 
  * Для текстов используются одномерные свёртки.
  * Хорошо подходят для нахождения в данных локальных паттернов.
  * Эффективно распараллеливаются на видеокартах 
  * Достаточно быстрые и простые 
  * Хорошо учатся 
  * Недостаточно гибкие и мощные, для широких паттернов — например, сравнивать первое слово в предложении и последнее, игнорируя при этом слова между ними. А также, чтобы увеличить максимальную длину паттерна, нужно существенно увеличить количество параметров свёрточной сети.

2. **Рекуррентные блоки**
  * Последовательно токен за токеном
  * Информация обо всём предложении
  * В результате могут учитываться достаточно длинные зависимости. 
  * Учить гораздо сложнее.

3. **Блоки пулинга**
  * убирает мелкие детали и оставляет только значимые. 
  * соседние элементы матриц усредняются или заменяются на один — например, максимальный.

4. **Блоки внимания**
  * По сути, механизм внимания осуществляет попарное сравнение элементов двух последовательностей, и позволяет выбрать только наиболее значимые их элементы, чтобы продолжить работу только с ними, а всё остальное убрать. 
  * Можно рассматривать механизм внимания как умный адаптивный пулинг. 
  * Сети "с вниманием", как правило, хорошо учатся. 

5. **Рекурсивные нейросети**
  * обобщение RNN
  * работают не с последовательностями, а с деревьями. 
  * Они применяются, например, для того, чтобы сначала выполнить синтаксический анализ, а потом пройтись по построенному дереву и агрегировать информацию из отдельных узлов. 

6. **Графовые свёрточные нейросети**
  * Обобщение и свёрточных, и рекуррентных нейросетей на произвольную структуру графа. 
  * Сейчас активно исследуется.

### Общее описание сверток

Напомню, свертки - это то, с чего начался хайп нейронных сетей в районе 2012-ого.

Работают они примерно так:  
![Conv example](https://image.ibb.co/e6t8ZK/Convolution.gif)   
From [Feature extraction using convolution](http://deeplearning.stanford.edu/wiki/index.php/Feature_extraction_using_convolution).

Формально - учатся наборы фильтров, каждый из которых скалярно умножается на элементы матрицы признаков. На картинке выше исходная матрица сворачивается с фильтром
$$
 \begin{pmatrix}
  1 & 0 & 1 \\
  0 & 1 & 0 \\
  1 & 0 & 1
 \end{pmatrix}
$$

Но нужно не забывать, что свертки обычно имеют ещё такую размерность, как число каналов. Например, картинки имеют обычно три канала: RGB.  
Наглядно демонстрируется как выглядят при этом фильтры [здесь](http://cs231n.github.io/convolutional-networks/#conv).

После сверток обычно следуют pooling-слои. Они помогают уменьшить размерность тензора, с которым приходится работать. Самым частым является max-pooling:  
![maxpooling](http://cs231n.github.io/assets/cnn/maxpool.jpeg)  
From [CS231n Convolutional Neural Networks for Visual Recognition](http://cs231n.github.io/convolutional-networks/#pool)

### Свёртки для текстов

Для текстов свертки работают как n-граммные детекторы (примерно). Каноничный пример символьной сверточной сети:

![text-convs](https://image.ibb.co/bC3Xun/2018_03_27_01_24_39.png)  
From [Character-Aware Neural Language Models](https://arxiv.org/abs/1508.06615)

*Сколько учится фильтров на данном примере?*

На картинке показано, как из слова извлекаются 2, 3 и 4-граммы. Например, желтые - это триграммы. Желтый фильтр прикладывают ко всем триграммам в слове, а потом с помощью global max-pooling извлекают наиболее сильный сигнал.

Что это значит, если конкретнее?

Каждый символ отображается с помощью эмбеддингов в некоторый вектор. А их последовательности - в конкатенации эмбеддингов.  
Например, "abs" $\to [v_a; v_b; v_s] \in \mathbb{R}^{3 d}$, где $d$ - размерность эмбеддинга. Желтый фильтр $f_k$ имеет такую же размерность $3d$.  
Его прикладывание - это скалярное произведение $\left([v_a; v_b; v_s] \odot f_k \right) \in \mathbb R$ (один из желтых квадратиков в feature map для данного фильтра).

Max-pooling выбирает $max_i \left( [v_{i-1}; v_{i}; v_{i+1}] \odot f_k \right)$, где $i$ пробегается по всем индексам слова от 1 до $|w| - 1$ (либо по большему диапазону, если есть padding'и).   
Этот максимум соответствует той триграмме, которая наиболее близка к фильтру по косинусному расстоянию.

В результате в векторе после max-pooling'а закодирована информация о том, какие из n-грамм встретились в слове: если встретилась близкая к нашему $f_k$ триграмма, то в $k$-той позиции вектора будет стоять большое значение, иначе - маленькое.

А учим мы как раз фильтры. То есть сеть должна научиться определять, какие из n-грамм значимы, а какие - нет.

В примере выше мы рассмотрели посимвольное применение свёрток к тексту. Аналогичный подход применим и к токенам на уровне слов. ![text-conv](https://i.ibb.co/pzpLGjD/screen-1.png)
Каждое слово представленно эмбедингом. Свёртки тут выполняют роль биграм.

Какой размер ядра свёртки?

![text-conv2](https://i.ibb.co/wJsDJf1/screen-2.png)

На практике вместо того что бы использовать один фильтр, как правило используют несколько (биграммы, триграммы и тд) каждый из фильтров пораждает своё признаковое описание и даёт несколько больше информации о тексте чем один фильтр. В процессе обучения сети эти фильтры выучиваются и мы можем смотреть на различные взаимосвязи со словами внутри n-gram.

Есть ли проблемы в текущей архитектуре?

Как быть когда мы применили много фильтров и получили несколько выходов?

Эти проблемы решает пулинговый слой.
![text-pool](https://i.ibb.co/J5L1TWx/screen-3.png)
Для текста это max или avg по временной оси. Когда мы берём max-over-time pooling мы обучаем сеть на то что бы она брала максимально информативный фильтр на основе которого можно было бы сделать какие-то выводы. В количестве фильтров мы не ограничены поэтому можно брать большое количество фильтров и пытаться выбирать наиболее информативный фильтр на который обучится сеть.


Типичная схема классификации текста с импользованием свёрток выглядит следующим образом
![text-cls](https://i.ibb.co/N9WyTg0/screen-4.png)
Сначала идут эмбединги слов, затем свёрточные слои, затем Max over time pooling. Причём для задач текста как правило max over time pooling работает лучше чем avg.

Для повышения качества используют много различных свёрток и берётся не один max-pooling, а  K-max это тоже помогает в задачах.
![text-cls2](https://i.ibb.co/2ty181Y/screen-5.png)

Так же как и с изображениями для обработки текста есть глубокие свёрточные архитектуры

![deep-cls](https://i.ibb.co/cTLPPKs/screen-6.png)

Это ResNet подобная архитектура. Сеть довольно мощная и с помощью подобн№ых архитектур сетей можно решать различные задачи, не только классификацию.


# Практика Text classification using CNN

## Задача (Sentiment Analysis)

Собраны твиты 2-ух тональностей, необходимо произвести классификацию на 2-а класса.

In [ ]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [ ]:
import pandas as pd

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

In [ ]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [ ]:
df_test.head()

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."
3,204153,RT @RonyLiss: @colf_ne блин((\nа я шипперила Ф...
4,204154,"RT @anna_romt: @ZADROT_PO_IGRAM блин,каждое во..."


In [ ]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


### Предобработка

In [ ]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

/home/rzaharov@mvs.local/.local/share/virtualenvs/nlp-course-A3nIyBIP/lib/python3.8/site-packages/sklearn/datasets/lfw.py:178: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if img.ndim is 0:
[nltk_data] Downloading package punkt to
[nltk_data]     /home/rzaharov@mvs.local/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [ ]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [ ]:
tokens_filtered_top[:10]

['rt',
 'd',
 'хотеть',
 'знать',
 'ян',
 'мочь',
 'любить',
 'завтра',
 'мой',
 'хороший']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [ ]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [ ]:
x_train.shape

(181467, 40)

In [ ]:
max_len

40

In [ ]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1, 168,
         8], dtype=int32)

# Keras model

In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [ ]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
  1/319 [..............................] - ETA: 0s - loss: 0.6937 - accuracy: 0.5078WARNING:tensorflow:From /home/rzaharov@mvs.local/.local/share/virtualenvs/nlp-course-A3nIyBIP/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
319/319 [==============================] - 14s 44ms/step - loss: 0.6290 - accuracy: 0.6135 - val_loss: 0.6180 - val_accuracy: 0.6169
Epoch 2/20
319/319 [==============================] - 13s 41ms/step - loss: 0.6166 - accuracy: 0.6252 - val_loss: 0.6160 - val_accuracy: 0.6224
Epoch 3/20
319/319 [==============================] - 12s 39ms/step - loss: 0.6135 - accuracy: 0.6280 - val_loss: 0.6138 - val_accuracy: 0.6304
Epoch 4/20
319/319 [==============================] - 13s 41ms/step - loss: 0.6101 - accuracy: 0.6310 - val_loss: 0.6139 - val_accuracy: 0.6224


In [ ]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 7ms/step - loss: 0.6184 - accuracy: 0.6248


Test score: 0.6184109449386597
Test accuracy: 0.6248291730880737


In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 0s 7ms/step


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score 

In [ ]:
vect = TfidfVectorizer(ngram_range=(1, 2), analyzer='word', lowercase=False)

In [ ]:
train_ft = vect.fit_transform(df_train['text'])
valid_ft = vect.transform(df_val['text'])

In [ ]:
lgr = LogisticRegression()

In [ ]:
lgr.fit(train_ft, df_train['class'].to_numpy())

/home/rzaharov@mvs.local/.local/share/virtualenvs/nlp-course-A3nIyBIP/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = lgr.predict(valid_ft)

In [ ]:
accuracy_score(df_val['class'].to_numpy(), y_pred)

0.7362341841908037

In [ ]:
from gensim.models import Word2Vec

In [ ]:
df_train['text']

0         alisachachkaн уезжаааааааать ❤ тожена хотеть у...
1         rt galyginvadim ребята девчата кино любовь зав...
2           rt artemklyushin ктоненавидеть пробка ретвит rt
3         rt epupybobv хотеться котлета покиевск запретн...
4         karinekurganova yessboss босапоп есбосан боять...
                                ...                        
181462                     классный новый httptcolejaguxnwu
181463                       видеть человек привет игнорита
181464    julia69styles длинный диагноз вкратце аллергич...
181465    technoslav ух серенький кнопочень телефон дост...
181466    kris1d07 собираться сделатьd навешать фотка ва...
Name: text, Length: 181467, dtype: object

In [ ]:
modelW2V = Word2Vec(sentences=df_train['text'].apply(str.split), size=100, window=5, min_count=5, workers=8)

In [ ]:
modelW2V.wv.vocab

{'❤': <gensim.models.keyedvectors.Vocab at 0x7f7d606bb1f0>,
 'тожена': <gensim.models.keyedvectors.Vocab at 0x7f7d606bb790>,
 'хотеть': <gensim.models.keyedvectors.Vocab at 0x7f7c62c34f10>,
 'уезжать': <gensim.models.keyedvectors.Vocab at 0x7f7c62c34f70>,
 'rt': <gensim.models.keyedvectors.Vocab at 0x7f7c62c34880>,
 'ребята': <gensim.models.keyedvectors.Vocab at 0x7f7c62c346d0>,
 'девчата': <gensim.models.keyedvectors.Vocab at 0x7f7c62c34ca0>,
 'кино': <gensim.models.keyedvectors.Vocab at 0x7f7c62c34bb0>,
 'любовь': <gensim.models.keyedvectors.Vocab at 0x7f7c62c34ee0>,
 'завтра': <gensim.models.keyedvectors.Vocab at 0x7f7d7b8135e0>,
 'artemklyushin': <gensim.models.keyedvectors.Vocab at 0x7f7d7b813c40>,
 'ктоненавидеть': <gensim.models.keyedvectors.Vocab at 0x7f7d58143310>,
 'пробка': <gensim.models.keyedvectors.Vocab at 0x7f7d58143190>,
 'ретвит': <gensim.models.keyedvectors.Vocab at 0x7f7d58143340>,
 'хотеться': <gensim.models.keyedvectors.Vocab at 0x7f7c562c1040>,
 'котлета': <gensi

In [ ]:
vect_idf = TfidfVectorizer()
vect_idf.fit_transform(df_train['text'])
tfidf = dict(zip(vect_idf.get_feature_names(), vect_idf.idf_))

In [ ]:
tfidf

{'00': 9.32464497468466,
 '000': 8.71438545393048,
 '0000': 9.707637226940765,
 '00000': 12.01022231993481,
 '000000': 12.415687428042975,
 '0000000': 12.01022231993481,
 '00000001': 12.415687428042975,
 '000009': 12.01022231993481,
 '00000asrova': 12.415687428042975,
 '00008': 12.415687428042975,
 '0000ибо': 12.415687428042975,
 '0000нуль': 12.415687428042975,
 '0000пойти': 12.415687428042975,
 '0001': 12.415687428042975,
 '0005': 12.415687428042975,
 '0006': 12.415687428042975,
 '0009': 12.415687428042975,
 '000nana000': 11.49939669616882,
 '000к': 12.415687428042975,
 '000ноль': 12.415687428042975,
 '000ный': 12.415687428042975,
 '000рубль': 12.415687428042975,
 '000ть': 12.415687428042975,
 '001': 12.415687428042975,
 '0019': 12.415687428042975,
 '002': 12.415687428042975,
 '0024': 12.415687428042975,
 '003': 12.415687428042975,
 '0030': 11.49939669616882,
 '004anna': 12.415687428042975,
 '005': 12.415687428042975,
 '0050': 12.415687428042975,
 '0053': 12.415687428042975,
 '007': 1

In [ ]:
rt = vect_idf.vocabulary_.items()

In [ ]:
tfidf['alisachachka']

12.415687428042975

In [ ]:
vect_idf.idf_[vect_idf.vocabulary_['alisachachka']]

12.415687428042975

In [ ]:
len(tfidf)

256172

In [ ]:
from collections import defaultdict

In [ ]:
max_idf = max(vect_idf.idf_)

word2weight = defaultdict(
    lambda: max_idf,
    [(w, vect_idf.idf_[i]) for w, i in vect_idf.vocabulary_.items()])

In [ ]:
def get_vect_mean(txt):
    vector_w2v = np.zeros(100)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V:
            vector_w2v += modelW2V[wrd]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

def get_vect_idf(txt):
    vector_w2v = np.zeros(100)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V:
            iddf_ = tfidf.get(wrd, 1.)
            vector_w2v += modelW2V[wrd]*iddf_
            n_w2v += iddf_
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
arr_vect = []
for txt in tqdm_notebook(df_train['text']):
    arr_vect.append(get_vect_mean(txt))
    
arr_vect_valid = []
for txt in tqdm_notebook(df_val['text']):
    arr_vect_valid.append(get_vect_mean(txt))
    
train_w2v = np.asarray(arr_vect)    
valid_w2v = np.asarray(arr_vect_valid)

<ipython-input-61-196588226526>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for txt in tqdm_notebook(df_train['text']):


<ipython-input-59-e89a2fe47eca>:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if wrd in modelW2V:
<ipython-input-59-e89a2fe47eca>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_w2v += modelW2V[wrd]


<ipython-input-61-196588226526>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for txt in tqdm_notebook(df_val['text']):


In [ ]:
lgr_w2v = LogisticRegression()

In [ ]:
lgr_w2v.fit(train_w2v, df_train['class'].to_numpy())

/home/rzaharov@mvs.local/.local/share/virtualenvs/nlp-course-A3nIyBIP/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = lgr_w2v.predict(valid_w2v)

In [ ]:
accuracy_score(df_val['class'].to_numpy(), y_pred)

0.6555129392055724

In [ ]:
arr_vect = []
for txt in tqdm_notebook(df_train['text']):
    arr_vect.append(get_vect_idf(txt))
    
arr_vect_valid = []
for txt in tqdm_notebook(df_val['text']):
    arr_vect_valid.append(get_vect_idf(txt))
    
train_w2v = np.asarray(arr_vect)    
valid_w2v = np.asarray(arr_vect_valid)

<ipython-input-66-23a6bf44f63a>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for txt in tqdm_notebook(df_train['text']):


<ipython-input-59-e89a2fe47eca>:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if wrd in modelW2V:
<ipython-input-59-e89a2fe47eca>:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_w2v += modelW2V[wrd]*iddf_


<ipython-input-66-23a6bf44f63a>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for txt in tqdm_notebook(df_val['text']):


In [ ]:
lgr_w2v = LogisticRegression()
lgr_w2v.fit(train_w2v, df_train['class'].to_numpy())
y_pred = lgr_w2v.predict(valid_w2v)

/home/rzaharov@mvs.local/.local/share/virtualenvs/nlp-course-A3nIyBIP/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "


In [ ]:
accuracy_score(df_val['class'].to_numpy(), y_pred)

0.6500903760525504